In [1]:
import requests as rq
import pandas as pd 
import numpy as np 

## Creating the Psicube dataframe

In [2]:
url = 'http://pseudogene.org/psicube/data/psidr/psiDR-HUMAN-April2014.txt' 
r = rq.get(url) #request to get the url
open('pseudogene.txt', 'wb').write(r.content) #download url in current directory, save as "pseudogene.txt"
df = pd.read_csv('pseudogene.txt', sep = '\s+', engine='python', header=[16]) 

## Parsing Psicube for the resurrected pseudogenes

In [3]:
target = df.loc[df['Activity_features'] == 'Tnx_1:Pol2_1:AC_1:TF_1'].copy() #target values are the ones with the specfied activity feature
target.to_csv('targetdata.txt', index=None, sep='\t') 

## Mapping the Pseudo Pipe ID and Protein Families

In [6]:
ppipe = pd.read_csv('PPipe.txt', sep='\s+', engine='python', header=[0]) #information with the pseudopipe ID
pfam = pd.read_csv('martquery_0618213934_745.txt', header=[0])
ppipe = ppipe.rename(columns={"EnsemblID.version": "#Pseudogene_id"}) #changes Ensemble id version column to pseudogene id
pfam = pfam.rename(columns={'Transcript stable ID version' : 'Parent_transcript'}) #Changes Transcript ID to Parent Transcript
combined = pd.merge(target, ppipe, how='left', on="#Pseudogene_id")
combined = pd.merge(combined, pfam, on ='Parent_transcript', how='left') 
combined.to_csv('combined_data.txt', index=None, sep='\t') 
combined = combined.replace(np.nan, 'N/A', regex=True) #replaces NaN with N/A
combined

,#Pseudogene_id,Biotype,Parent_gene,Parent_transcript,Activity_features,PseudoPipeID,EnsemblID,Gene stable ID,Gene stable ID version,Transcript stable ID,Pfam ID,Protein stable ID,Protein stable ID version
0,ENST00000359901.3,DUP,.,.,Tnx_1:Pol2_1:AC_1:TF_1,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
1,ENST00000406724.1,DUP,.,.,Tnx_1:Pol2_1:AC_1:TF_1,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
2,ENST00000404404.1,DUP,.,.,Tnx_1:Pol2_1:AC_1:TF_1,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
3,ENST00000453917.1,PSSD,.,.,Tnx_1:Pol2_1:AC_1:TF_1,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
4,ENST00000448968.2,PSSD,.,.,Tnx_1:Pol2_1:AC_1:TF_1,PGOHUM00000291220,ENST00000448968,N/A,N/A,N/A,N/A,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,ENST00000377662.2,PSSD,ENSG00000112763.10,ENST00000312541.5,Tnx_1:Pol2_1:AC_1:TF_1,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
88,ENST00000492994.1,PSSD,ENSG00000058262.5,ENST00000243253.3,Tnx_1:Pol2_1:AC_1:TF_1,PGOHUM00000302420,ENST00000492994,N/A,N/A,N/A,N/A,N/A,N/A
89,ENST00000431295.1,PSSD,ENSG00000109475.12,ENST00000394668.2,Tnx_1:Pol2_1:AC_1:TF_1,PGOHUM00000296581,ENST00000431295,ENSG00000109475,ENSG00000109475.16,ENST00000394668,PF01199,ENSP00000378163,ENSP00000378163.2
90,ENST00000376334.3,DUP,ENSG00000109536.6,ENST00000226798.4,Tnx_1:Pol2_1:AC_1:TF_1,PGOHUM00000303876,ENST00000376334,N/A,N/A,N/A,N/A,N/A,N/A
